In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import altair as alt

In [2]:
#get data
#revenue sharing with athletes data (22% of each schools revenue, according to NIl they need to share w athletes)
shared_rev = pd.read_csv('all_schools_rev_sharing.csv')
#football teams get 77.3% of revenue sharing, according to https://nil-ncaa.com/
shared_rev['Football_Revenue_Share'] = shared_rev['22% of Revenue'] * 0.773


#copied html from https://247sports.com/season/2025-football/compositeteamrankings/ into a document, wouldn't let me access the website 
with open('tabletags.txt', 'r', encoding='utf-8') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

team_blocks = soup.find_all('li', class_='rankings-page__list-item')

teams_data = []

for team in team_blocks:
    rank = team.find('div', class_='primary').get_text(strip=True)
    team_name = team.find('div', class_='team').get_text(strip=True)
    commits_text = team.find('div', class_='total').get_text(strip=True)
    commits = commits_text.split()[0]  # "25 Commits" -> "25"
    avg_rating = team.find('div', class_='avg').get_text(strip=True)
    points = team.find('div', class_='points').get_text(strip=True)
    
    stars = team.find_all('ul', class_='star-commits-list')
    five_star = four_star = three_star = '0'
    
    if stars:
        star_numbers = stars[0].find_all('div')
        if len(star_numbers) >= 3:
            five_star = star_numbers[0].get_text(strip=True)
            four_star = star_numbers[1].get_text(strip=True)
            three_star = star_numbers[2].get_text(strip=True)
    
    teams_data.append({
        'Rank': rank,
        'Team': team_name,
        'Total Commits': commits,
        '5-Stars': five_star,
        '4-Stars': four_star,
        '3-Stars': three_star,
        'Average': avg_rating,
        'Points': points
    })

recruit = pd.DataFrame(teams_data)
csv_path = 'recruiting_rankings_2025.csv'
recruit.to_csv(csv_path, index=False)

merged_df = pd.merge(shared_rev, recruit, left_on='School', right_on='Team')

In [3]:
print("School Revenue Sharing Info")
display(shared_rev.head())
print("School Recruitment Ranking Info")
display(recruit.head())
print("Merged Data")
display(merged_df.head())


School Revenue Sharing Info


,School,Conference,Annual Revenue,22% of Revenue,2025 Cap,Football_Revenue_Share
0,Ohio State,Big Ten,187930323,41344671,20500000,3.195943e+07
1,Michigan,Big Ten,166204848,36565067,20500000,2.826480e+07
2,Texas,SEC,161316575,35489647,20500000,2.743350e+07
3,Alabama,SEC,143674059,31608293,20500000,2.443321e+07
4,Nebraska,Big Ten,142131011,31268822,20500000,2.417080e+07


School Recruitment Ranking Info


,Rank,Team,Total Commits,5-Stars,4-Stars,3-Stars,Average,Points
0,1,Texas,25,5,14,6,93.63,312.27
1,2,Georgia,27,5,20,2,93.03,304.64
2,3,Alabama,21,3,16,2,93.42,298.40
3,4,Ohio State,26,3,18,5,92.46,297.72
4,5,Oregon,20,3,16,1,94.37,296.10


Merged Data


,School,Conference,Annual Revenue,22% of Revenue,2025 Cap,Football_Revenue_Share,Rank,Team,Total Commits,5-Stars,4-Stars,3-Stars,Average,Points
0,Ohio State,Big Ten,187930323,41344671,20500000,3.195943e+07,4,Ohio State,26,3,18,5,92.46,297.72
1,Michigan,Big Ten,166204848,36565067,20500000,2.826480e+07,6,Michigan,24,2,15,7,92.18,290.87
2,Texas,SEC,161316575,35489647,20500000,2.743350e+07,1,Texas,25,5,14,6,93.63,312.27
3,Alabama,SEC,143674059,31608293,20500000,2.443321e+07,3,Alabama,21,3,16,2,93.42,298.40
4,Nebraska,Big Ten,142131011,31268822,20500000,2.417080e+07,22,Nebraska,20,0,12,8,89.65,237.53


In [4]:

chart = alt.Chart(merged_df).mark_point(shape='circle', filled=True,size=100).encode(
    x=alt.X('Rank:Q', sort='ascending', scale=alt.Scale(domain=[0, 50])), 
    y='Football_Revenue_Share:Q',
    color='Conference:N',
    size='Total Commits:Q', 
    tooltip=['School', alt.Tooltip('Football_Revenue_Share:Q', title='Football Revenue Share', format="$,.0f"), 'Rank', 'Conference', 'Total Commits']
).properties(
    title='Relationship between Recruitment Ranking and Football Revenue Share',
	width=500
)

chart


alt.Chart(...)

In [7]:
brush = alt.selection_interval()

scatter = alt.Chart(merged_df).mark_point(filled=True, shape='circle', size=100).encode(
    x=alt.X('Rank:Q', sort='ascending', scale=alt.Scale(domain=[0, 50])),
    y='Football_Revenue_Share:Q',
    color='Conference:N',
    size='Total Commits:Q',
    tooltip=[
        'School',
        alt.Tooltip('Football_Revenue_Share:Q', title='Football Revenue Share', format='$,.0f'),
        'Rank', 'Conference', 'Total Commits'
    ]
).add_params(
    brush
).properties(
    width=500,
    height=400,
    title='Recruitment Rank vs Football Revenue Share'
)


def star_bar(star_type: str, title: str):
    return alt.Chart(merged_df).mark_bar().encode(
        y=alt.Y('School:N', sort=alt.EncodingSortField(field='Rank', order='ascending'), title=''),
        x=alt.X(f'{star_type}:Q', title=''),
        color=alt.Color('Conference:N',scale=alt.Scale(scheme='yellowgreenblue')),
        tooltip=['School', f'{star_type}', 'Conference']
    ).transform_filter(
        brush
    ).properties(
        height=100,
        title=title
    )


bars_5 = star_bar('5-Stars', '5-Star Recruits')
bars_4 = star_bar('4-Stars', '4-Star Recruits')
bars_3 = star_bar('3-Stars', '3-Star Recruits')


bars_combined = alt.vconcat(bars_5, bars_4, bars_3).resolve_scale(x='independent')


final_chart = alt.hconcat(scatter, bars_combined)

final_chart

alt.HConcatChart(...)

In [6]:
final_chart.save('recruitment.html')